In [ ]:
class_mapping ={
    'L10-1': 0,
    'L18- 1': 1,
    'L2-1': 2,
    'L4-1': 3,
    'L5-1': 4,
    'L6-1b': 5,
    'L7-2': 6,
    'RGA-1': 7,
    'RGA-4': 8,
    'ZigZag Marking': 9
}
class_mapping_1 = {
    "L1-1a": 0,
    "L1-1b": 1,
    "L1-1c": 2,
    "L2-1a": 3,
    "L2-1b": 4,
    "L2-1c": 5,
    "L3-1a": 6,
    "L3-1b": 7,
    "L3-2a": 8,
    "L3-2b": 9,
    "L4-1a": 10,
    "L4-1b": 11,
    "L4-1c": 12,
    "L4-1d": 13,
    "L5-1a": 14,
    "L5-1b": 15,
    "L5-1c": 16,
    "L6-1a": 17,
    "L6-1b": 18,
    "L6-1c": 19,
    "L6-1d": 20,
    "L7-1": 21,
    "L7-2": 22,
    "L8-1": 23,
    "L9-1a": 24,
    "L9-1b": 25,
    "L9-1c": 26,
    "L9-1d": 27,
    "L10-1": 28,
    "L11-1": 29,
    "L18- 1": 30,
    "L21-1": 31,
    "L24-1": 32,
    "L25-1": 33,
    "L26-1": 34,
    "L27-1": 35,
    "L36-1a": 36,
    "L36-1b": 37,
    "L36-1c": 38
}



In [ ]:
import os
import glob
import json
from PIL import Image
import shutil
import random

def convert_to_yolo(json_path, output_dir, image_dir, class_mapping):
    os.makedirs(output_dir, exist_ok=True)
    
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    # Extract image path
    image_path = os.path.join(image_dir, os.path.basename(data['imagePath']))
    
    # Check if the image exists
    if not os.path.exists(image_path):
        print(f"Warning: Image {image_path} not found. Skipping.")
        return
    
    # Get image dimensions
    with Image.open(image_path) as img:
        image_width, image_height = img.size
    
    # Process annotations in the "shapes" key
    for annotation in data['shapes']:
        label = annotation['label']
        points = annotation['points']
        
        # Skip labels not in the mapping
        if label not in class_mapping:
            continue
        class_id = class_mapping[label]
        
        # Normalize polygon points
        normalized_points = [
            [x / image_width, y / image_height] for x, y in points
        ]
        
        # Flatten points into YOLOv8 format
        flattened_points = " ".join(f"{x} {y}" for x, y in normalized_points)
        yolo_line = f"{class_id} {flattened_points}"
        
        # Write to label file
        shutil.copy(image_path, output_dir)
        image_name = os.path.splitext(os.path.basename(image_path))[0]
        label_file = os.path.join(output_dir, f"{image_name}.txt")
        with open(label_file, 'a') as f:
            f.write(yolo_line + "\n")

# Get all JSON files
allfiles = glob.glob(r'C:\Users\fbpza\Desktop\Road-Lines-Detection\KSA dataset Insp\Annotations' + os.sep + '*.json')

# Shuffle the files
random.shuffle(allfiles)

# Split into 80% train and 20% val
split_index = int(len(allfiles) * 0.8)
train_files = allfiles[:split_index]
val_files = allfiles[split_index:]

# Define output directories
train_output_dir = r'C:\Users\fbpza\Desktop\Road-Lines-Detection\KSA dataset Insp\final\train'
val_output_dir = r'C:\Users\fbpza\Desktop\Road-Lines-Detection\KSA dataset Insp\final\val'
image_dir = r'C:\Users\fbpza\Desktop\Road-Lines-Detection\KSA dataset Insp\Annotations'

# Create directories if they don't exist
os.makedirs(train_output_dir, exist_ok=True)
os.makedirs(val_output_dir, exist_ok=True)

# Process training files
for fileee in train_files:
    convert_to_yolo(
        json_path=fileee,
        output_dir=train_output_dir,
        image_dir=image_dir,
        class_mapping=class_mapping
    )

# Process validation files
for fileee in val_files:
    convert_to_yolo(
        json_path=fileee,
        output_dir=val_output_dir,
        image_dir=image_dir,
        class_mapping=class_mapping
    )



In [6]:
import os,glob
import json
from PIL import Image


class_mapping = {
    'L10-1': 0,
    'L18- 1': 1,
    'L2-1': 2,
    'L4-1': 3,
    'L5-1': 4,
    'L6-1b': 5,
    'L7-2': 6,
    'RGA-1': 7,
    'RGA-4': 8,
    'ZigZag Marking': 9
}

def convert_to_json(txt_path, output_dir, image_dir, class_mapping):
    os.makedirs(output_dir, exist_ok=True)
    
    # Read the YOLOv8 format file
    with open(txt_path, 'r') as f:
        lines = f.readlines()
    
    # Extract image name from txt file name
    image_name = os.path.splitext(os.path.basename(txt_path))[0]
    image_path = os.path.join(image_dir, f"{image_name}_result.jpg")
    
    # Check if the image exists
    if not os.path.exists(image_path):
        print(f"Warning: Image {image_path} not found. Skipping.")
        return
    
    # Get image dimensions
    with Image.open(image_path) as img:
        image_width, image_height = img.size
    
    # Prepare JSON structure
    json_data = {
        "imagePath": image_name + ".jpg",
        "shapes": []
    }
    
    # Process each line in the YOLOv8 format file
    for line in lines:
        parts = line.strip().split()
        class_id = int(parts[0])
        points = list(map(float, parts[1:]))
        
        # Denormalize points
        denormalized_points = [
            [points[i] * image_width, points[i + 1] * image_height] for i in range(0, len(points), 2)
        ]
        
        # Get label from class mapping
        label = [k for k, v in class_mapping.items() if v == class_id][0]
        
        # Add annotation to JSON structure
        json_data["shapes"].append({
            "label": label,
            "points": denormalized_points
        })
    
    # Save JSON file
    json_output_path = os.path.join(output_dir, f"{image_name}.json")
    with open(json_output_path, 'w') as f:
        json.dump(json_data, f, indent=4)

files_dir = r'C:\Users\fbpza\Desktop\Road-Lines-Detection\Data\Detected\images'
allfiles = glob.glob(files_dir + os.sep + '*.txt')


# Process training files
for fileee in allfiles:
    convert_to_json(fileee, files_dir, files_dir, class_mapping
    )
